## Deep Learning 

Using DL method to classify a set of news articles into real and fake news.

### Environment Setup

Importing necessary libraries, including BERT. Reading input files containting the real and fake news datasets.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import Dataset, DatasetDict
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# dataset_true = load_dataset("csv", data_files="/Users/sanaketabchi/project/DS/NLP/True.csv")
# dataset_fake = load_dataset("csv", data_files="/Users/sanaketabchi/project/DS/NLP/Fake.csv")

# dataset = concatenate_datasets([dataset_true['train'], dataset_fake['train']])

# dataset = dataset.train_test_split(test_size=0.3)


# train_df = pd.DataFrame(dataset["train"][:])

df_true = pd.read_csv("/Users/sanaketabchi/project/DS/NLP/True.csv")
df_fake = pd.read_csv("/Users/sanaketabchi/project/DS/NLP/Fake.csv")

# df_true.shape
# df_fake.shape

# df_true.head()

df_true['TrueNews?'] = [True for x in range(len(df_true.index))]
df_fake['TrueNews?'] = [False for x in range(len(df_fake.index))]

df_all = pd.concat([df_true, df_fake], axis = 0)

trainDS, testDS = train_test_split(df_all, test_size=0.3)

tds = Dataset.from_pandas(trainDS)
sds = Dataset.from_pandas(testDS) 
ds = DatasetDict()

ds['train'] = tds
ds['test'] = sds

ds




DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'subject', 'date', 'TrueNews?', '__index_level_0__'],
        num_rows: 31428
    })
    test: Dataset({
        features: ['title', 'text', 'subject', 'date', 'TrueNews?', '__index_level_0__'],
        num_rows: 13470
    })
})

In [2]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_ckpt)

Combining the real and fake news datasets into one dataframe.

In [3]:
# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

# def tokenize_title(batch):
#     return tokenizer(batch["title"], padding=True, truncation=True)

# Tokenize entire dataset 
tokenized_dataset = ds.map(tokenize, batched=True, batch_size=None)
# tokenized_dataset = tokenized_dataset_1.map(tokenize_title, batched=True, batch_size=None)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
num_labels = 2
model = TFAutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                             num_labels=num_labels)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [5]:
batch_size=64
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [6]:
tokenized_dataset["train"]

Dataset({
    features: ['title', 'text', 'subject', 'date', 'TrueNews?', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 31428
})

In [8]:
# Create tf datasets
tokenized_dataset["train"]
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset( 
    # columns=['title', 'text', 'subject', 'date', 'TrueNews?', '__index_level_0__', 'input_ids', 'attention_mask'],
    columns=['text',"TrueNews?", '__index_level_0__', 'input_ids', 'attention_mask'],
    label_cols=["TrueNews?"], 
    shuffle=True, 
    batch_size=batch_size,
    # collate_fn=data_collator
)
tf_valid_dataset = tokenized_dataset["test"].to_tf_dataset(
    # columns=['title', 'text', 'subject', 'date', 'TrueNews?', '__index_level_0__', 'input_ids', 'attention_mask'], 
    columns=["TrueNews?", '__index_level_0__', 'input_ids', 'attention_mask'],
    label_cols=["TrueNews?"], 
    shuffle=False, 
    batch_size=batch_size,
    # collate_fn=data_collator
)

In [9]:
tf_train_dataset

<PrefetchDataset element_spec=({'text': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'TrueNews?': TensorSpec(shape=(None,), dtype=tf.int64, name=None), '__index_level_0__': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [10]:
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import losses

In [11]:
# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=5e-5),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=metrics.SparseCategoricalAccuracy()
)

In [12]:
# Fit the model
model.fit(tf_train_dataset, 
          validation_data=tf_valid_dataset, 
          epochs=5)

Epoch 1/5


2023-01-18 21:05:45.181438: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


TypeError: in user code:

    File "/Users/sanaketabchi/miniforge3/envs/DS39/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/sanaketabchi/miniforge3/envs/DS39/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/sanaketabchi/miniforge3/envs/DS39/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/sanaketabchi/miniforge3/envs/DS39/lib/python3.9/site-packages/transformers/modeling_tf_utils.py", line 1495, in train_step
        y_pred = self(x, training=True)
    File "/Users/sanaketabchi/miniforge3/envs/DS39/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/nn/08g9ycbd6